In [1]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.callbacks import EarlyStopping

In [2]:
training_data_gen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range = 0.12,
    height_shift_range = 0.12,
    width_shift_range = 0.12,
    horizontal_flip = True
)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [3]:
train_generator = training_data_gen.flow_from_directory('./data/images/Images',
                                                        target_size=(300, 300),
                                                        batch_size=32,
                                                        class_mode='categorical')
test_generator = test_data_gen.flow_from_directory('./data/test',
                                                        target_size=(300, 300),
                                                        batch_size=32,
                                                        class_mode='categorical')

Found 16666 images belonging to 120 classes.
Found 3911 images belonging to 120 classes.


In [4]:
base_model = keras.applications.ResNet50(
    weights='imagenet',
    input_shape=(300,300, 3),
    include_top=False)

for layer in base_model.layers:
    layer.trainable = False

In [5]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(120, activation='softmax')(x)

model = Model(base_model.input, predictions)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 306, 306, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 150, 150, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [6]:
model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [7]:
early_stopping = EarlyStopping(
    monitor='accuracy',
    patience = 1,
    restore_best_weights = True
)
model.fit(train_generator,
                        epochs=10,
                        validation_data=test_generator, callbacks=[early_stopping])

Epoch 1/10
521/521 [==============================] - 3648s 7s/step - loss: 4.8022 - accuracy: 0.0088 - val_loss: 4.7879 - val_accuracy: 0.0113
Epoch 2/10
521/521 [==============================] - 3667s 7s/step - loss: 4.7874 - accuracy: 0.0118 - val_loss: 4.7877 - val_accuracy: 0.0123
Epoch 3/10
521/521 [==============================] - 2006s 4s/step - loss: 4.7850 - accuracy: 0.0112 - val_loss: 4.7837 - val_accuracy: 0.0107


In [8]:
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print(f'\nTest accuracy: {test_acc}')

123/123 - 352s - loss: 4.7877 - accuracy: 0.0123 - 352s/epoch - 3s/step

Test accuracy: 0.012273075990378857


In [9]:
model.save('sanford_dog_resnet.h5')